In [351]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [184]:
# read in all the words
words = open('data/names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])
# build the vocabulary of characters and mappings to/from integers


chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)
print(f'Character to index mapping: {itos}')
print(f'Vocabulary size: {vocab_size}')

# Shuffle the words
random.seed(42)
random.shuffle(words)


32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
Character to index mapping: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Vocabulary size: 27


In [245]:
# Function to encode words into a single continuous sequence of indices
def encode_words(words):
    encoded = []
    for w in words:
        encoded.extend([stoi[ch] for ch in '.' + w + '.'])
    return encoded

# Encode all words into a single sequence
encoded = encode_words(words)

# Function to create input-output pairs from a sequence
def create_pairs(seq, block_size):
    X, Y = [], []
    for i in range(0, len(seq) - block_size):
        X.append(seq[i:i+block_size])
        Y.append(seq[i+1:i+block_size+1])
    X = torch.tensor(X, dtype=torch.long)
    Y = torch.tensor(Y, dtype=torch.long)
    return X, Y

# Split into training and development sets
n = len(encoded)
n1 = int(0.8 * n)

train_seq = encoded[:n1]
dev_seq = encoded[n1:]

block_size = 8

# Create pairs for training and development sets
Xtr, Ytr = create_pairs(train_seq, block_size)
Xdev, Ydev = create_pairs(dev_seq, block_size)

print(f'Training data shapes - X: {Xtr.shape}, Y: {Ytr.shape}')
print(f'Development data shapes - X: {Xdev.shape}, Y: {Ydev.shape}')

# Function to split data into batches of shape (batch_size, time_steps, -1)
def split_into_batches(X, Y, batch_size):
    # Ensure the data size is a multiple of batch_size
    num_batches = X.size(0) // batch_size
    X = X[:num_batches * batch_size]
    Y = Y[:num_batches * batch_size]
    
    # Reshape into batches
    X = X.view(batch_size, -1, X.size(-1))
    Y = Y.view(batch_size, -1, Y.size(-1))
    return X, Y

# Example output
print(f'Training batches shape - X: {Xtr.shape}, Y: {Ytr.shape}')
print(f'Development batches shape - X: {Xdev.shape}, Y: {Ydev.shape}')


Training data shapes - X: torch.Size([208135, 8]), Y: torch.Size([208135, 8])
Development data shapes - X: torch.Size([52028, 8]), Y: torch.Size([52028, 8])
Training batches shape - X: torch.Size([208135, 8]), Y: torch.Size([208135, 8])
Development batches shape - X: torch.Size([52028, 8]), Y: torch.Size([52028, 8])


In [246]:
print(Xtr[2])
print(Ytr[2])

tensor([21,  8,  5, 14,  7,  0,  0,  4])
tensor([ 8,  5, 14,  7,  0,  0,  4,  9])


In [247]:
one_hot_Xtr = F.one_hot(Xtr, 27)
one_hot_Xtr = one_hot.view(8, -1, 27)

In [248]:
Xb.shape

torch.Size([8, 32, 27])

In [249]:
Yb.shape

torch.Size([8, 32])

In [251]:
batch_size = 32
batch = torch.randint(0, one_hot_Xtr.shape[1], (batch_size,))
Xb = one_hot_Xtr[:, batch, :] # (8, 32, 27)
Yb = Ytr[batch]
Yb = Yb.view(8, -1)
time_steps, batch_size, input_size = Xb.shape
hidden_size = 30

torch.Size([8, 32, 27])

In [347]:
# for t in time_steps:

Xb = Xb.type(torch.float32)
Yb = Yb.type(torch.int64)

hidden = 30

# Parameters
WLSTM1 = torch.randn(vocab_size, hidden_size)
WLSTM2 = torch.randn(hidden_size, hidden_size)
Fo = torch.randn(batch_size, hidden_size)
i1 = torch.randn(batch_size, hidden_size)
i2 = torch.randn(batch_size, hidden_size)
O = torch.randn(batch_size, hidden_size)
bias1 = torch.zeros(hidden_size)
bias2 = torch.zeros(hidden_size)
bias3 = torch.zeros(hidden_size)
bias4 = torch.zeros(hidden_size)
output_matrix = torch.randn(hidden_size, vocab_size)

# Storage
# prev = torch.zeros(time_steps, batch_size, hidden_size)
hidden1 = torch.zeros(time_steps, batch_size, hidden_size)
hidden2 = torch.zeros(time_steps, batch_size, hidden_size)
total = torch.zeros(time_steps, batch_size, hidden_size)
preact1 = torch.zeros(time_steps, batch_size, hidden_size)
preact2 = torch.zeros(time_steps, batch_size, hidden_size)
preact3 = torch.zeros(time_steps, batch_size, hidden_size)
preact4 = torch.zeros(time_steps, batch_size, hidden_size)
act1 = torch.zeros(time_steps, batch_size, hidden_size)
act2 = torch.zeros(time_steps, batch_size, hidden_size)
act3 = torch.zeros(time_steps, batch_size, hidden_size)
act4 = torch.zeros((time_steps, batch_size, hidden_size))
C = torch.zeros((time_steps, batch_size, hidden_size))
Ct = torch.zeros((time_steps, batch_size, hidden_size))
Hout = torch.zeros((time_steps, batch_size, hidden_size))
logits = torch.zeros((time_steps, batch_size, vocab_size))
c0 = torch.zeros(batch_size, hidden_size)
h0 = torch.zeros((batch_size, hidden_size))


In [262]:
Yb[0]

tensor([ 0,  0,  1, 12,  9,  3,  9, 15, 19,  1,  2, 18,  9, 14,  1,  0, 15, 18,
         9,  1, 14, 14,  1,  0, 21, 18,  0,  0,  2, 18,  1, 25])

In [269]:
logprobs[0].shape

torch.Size([32, 27])

In [353]:
# Forward pass
loss = 0
for t in range(time_steps):
    prevh = H[t-1] if t > 0 else h0
    prevc = C[t-1] if t > 0 else c0
    # 32 examples of 27 one hot encoded words
    # (32, 27)
    # (32, 30)
    
    # hidden1[t] = Xb[t] @ WLSTM1 # #(32, 27) @ (27, 30) = (32, 30)
    # hidden2[t] = prevh @ WLSTM2 # (32, 30)
    # total[t] = hidden1[t] + hidden2[t] # (32, 30)
    
    preact1[t] = Xb[t] @ Fhh + prevh  bias1 # (32, 30)
    preact2[t] = Xb[t] @ i1 + prevh bias2 # (32, 30)
    preact3[t] = Xb[t] @ i2 + prevh bias3 # (32, 30)
    preact4[t] = Xb[t] @ Oih  prevh * Ohh # (32, 30)
    
    act1[t] = torch.sigmoid(preact1[t]) # (32, 30)
    act2[t] = torch.sigmoid(preact2[t]) # (32, 30)
    act3[t] = torch.tanh(preact3[t]) # (32, 30)
    act4[t] = torch.sigmoid(preact4[t]) # (32, 30)
    
    C[t] = act2[t] * prevc + act1[t] * act4[t] # (32, 30)
    Ct[t] = torch.tanh(C[t]) # (32, 30)
    Hout[t] = Ct[t] * act3[t] # (32, 30)
    
    logits[t] = Hout[t] @ output_matrix # (32, 27)
    
    counts = logits.exp()
    counts_sum = counts.sum(1, keepdims=True)
    counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
    probs = counts * counts_sum_inv
    logprobs = probs.log()
    loss += -logprobs[t][torch.arange(32), Yb].mean()


print (loss / time_steps)

tensor(3.7705)


In [355]:
# # Backward pass

# # To update
# dWLSTM1 = torch.zeros(vocab_size, hidden_size) #(27, 30)
# dWLSTM2 = torch.zeros(hidden_size, hidden_size) # (30, 30)

# dFo = torch.zeros(batch_size, hidden_size) # (32, 30)
# di1 = torch.zeros(batch_size, hidden_size) # (32, 30)
# di2 = torch.zeros(batch_size, hidden_size) # (32, 30)
# dO = torch.zeros(batch_size, hidden_size) # (32, 30)

# dbias1 = torch.zeros(hidden_size) # (30)
# dbias2 = torch.zeros(hidden_size) # (30)
# dbias3 = torch.zeros(hidden_size) # (30)
# dbias4 = torch.zeros(hidden_size) # (30)

# doutput_matrix = torch.randn(hidden_size, vocab_size) # (30, 27)

# # Placeholders (indexed with t)
# dlogits = torch.zeros((time_steps, batch_size, vocab_size)) # (30, 27)
# dhidden1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
# dhidden2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
# dtotal = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)

# dpreact1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30) 
# dpreact2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
# dpreact3 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
# dpreact4 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)

# dact1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30) 
# dact2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
# dact3 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
# dact4 = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)

# dC = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
# dCt = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
# dHout = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
# dlogits = torch.zeros((time_steps, batch_size, vocab_size)) # (32, 27)

# dc0 = torch.zeros(batch_size, hidden_size)
# dh0 = torch.zeros((batch_size, hidden_size))



# # Backpropogate cross entropy
# dlogits[t] = F.softmax(logits[t], 1)
# dlogits[t][torch.arange(batch_size), Yb] -= 1
# dlogits /= n

# # Backpropogate dHout
# dHout[t] =  dlogits[t] @ output_matrix.T # (32, 27) @ (27, 30) = (32, 30) dHt on paper derivation

# # Backpropogate output matrix
# doutput_matrix = dlogits[t].T @ dHout[t] # (32, 27)




# # 1. Backpropogate dact3 (output gate)
# dact3[t] = dHout[t] * Ct[t] # (32, 27) * (32, 27) = (32, 27)

# # 2. Backpropogate dC (current cell state)
# dC[t] = dHout[t] * act3[t] * (1 - torch.tanh(C[t])**2) # (32, 27) * (32, 27) * (32, 27) = (32, 27)


# # 3. Backpropogate dF for act2 and previous cell state
# if t > 0:
#     # Forget gate activations
#     # Last cell activations
#     dact1[t] = dHout[t] * C[t-1] # (32, 27) * (32, 27) = (32, 27)
#     dC[t-1] = dCt[] * Fo
# else:
#     df = dHout[t] * C[t-1]
#     dc0 = dC[t] * Fo

# # Backpropogate i1 activations
# dact2[t] = dC[t] * i2

# # Backpropogate i2 activations
# dact3[t] = dC[t] * i1

# # Backpropogate all preactivations
# dpreact1[t] = dact1[t] * act1[t] * ( 1- act1[t])
# dpreact2[t] = dact2[t] * act2[t] * ( 1- act2[t])
# dpreact3[t] = dact3[t] * (1 - torch.tanh(dpreact3[t])**2)
# dpreact4[t] = dact4[t] * act4[t] * ( 1- act4[t])

# # Backpropogate gates
# dFo = dpreact1[t] * total[t]
# di1 = dpreact2[t] * total[t]
# di2 = dpreact3[t] * total[t]
# dO = dpreact4[t] * total[t]

# # Backpropogate total
# dtotal[t] = dpreact1[t] * Fo + dpreact2[t] * i1 + dpreact3[t] * i2 + dpreact4[t] * O

# # Backpropogate hidden
# dhidden1 = 1 * dtotal[t]
# dhidden2 = 1 * dtotal[t]

# # Backpropogate inputs
# dprevh

#     prevh = H[t-1] if t > 0 else h0
#     prevc = C[t-1] if t > 0 else c0
#     # 32 examples of 27 one hot encoded words
#     hidden1[t] = Xb[t] @ WLSTM1 # (32, 27) @ (27, 30) = (32, 30)
#     hidden2[t] = prevh @ WLSTM2 # (32, 30)
#     total[t] = hidden1[t] + hidden2[t] # (32, 30)

#     preact1[t] = total[t] * Fo + bias1 # (32, 30)
#     preact2[t] = total[t] * i1 + bias2 # (32, 30)
#     preact3[t] = total[t] * i2 + bias3 # (32, 30)
#     preact4[t] = total[t] * O + bias4 # (32, 30)
    
#     act1[t] = torch.sigmoid(preact1[t]) # (32, 30)
#     act2[t] = torch.sigmoid(preact2[t]) # (32, 30)
#     act3[t] = torch.tanh(preact3[t]) # (32, 30)
#     act4[t] = torch.sigmoid(preact4[t]) # (32, 30)

#     C[t] = act2[t] * prevc + act1[t] * act4[t] # (32, 30)

#     Ct[t] = torch.tanh(C[t]) # (32, 30)
#     Hout[t] = Ct[t] * act3[t] # (32, 30)

    